In [1]:
!pip install nilearn==0.9.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
import numpy as np
import datetime
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import matplotlib
from matplotlib import pyplot as plt
from nilearn import datasets
from nilearn import plotting
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision.models.feature_extraction import create_feature_extractor, get_graph_node_names
from torchvision import transforms
from sklearn.decomposition import IncrementalPCA
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr as corr
import h5py

import tensorflow_datasets as tfds
import tensorflow as tf
from keras import datasets, Sequential
from keras.layers import Conv2D, Dense, MaxPooling2D, Flatten
from sklearn.metrics import confusion_matrix
from google.colab import drive

In [3]:
!git clone https://github.com/fsegatz01/IANNwTF_Group16.git

%cd IANNwTF_Group16/project/library

fatal: destination path 'IANNwTF_Group16' already exists and is not an empty directory.
/content/IANNwTF_Group16/project/library


In [ ]:
# our own libraries

import model_library as ml
import data_processing_library as dpl

In [ ]:
### CONNECTION TO GOOGLE DRIVE ###
drive.mount('/content/drive', force_remount=True)
os.chdir ("drive/MyDrive")


##CIFAR

In [ ]:
########## LOADING CIFAR DATASET #############

cifar, cifar_info = tfds.load("cifar100", as_supervised=True, with_info =True)

# split data into train and test
train_ds = cifar["train"]
val_ds = cifar["test"]


train_cifar = dpl.preprocess_cifar(train_ds, 'cifar', 32)
test_cifar = dpl.preprocess_cifar(val_ds, 'cifar', 32)



Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar100/3.0.2.incompleteOP67WU/cifar100-train.tfrecord*...:   0%|        …

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/cifar100/3.0.2.incompleteOP67WU/cifar100-test.tfrecord*...:   0%|         …

Dataset cifar100 downloaded and prepared to /root/tensorflow_datasets/cifar100/3.0.2. Subsequent calls will reuse this data.


NameError: ignored

In [ ]:
### CIFAR ON CNN ###

# set up the model
cnn_cifar_model = ml.ConvModel()

cnn_cifar_model.compile(loss='categorical_crossentropy',
                        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                        metrics=['categorical_accuracy'])


# train the model
history = cnn_cifar_model.fit(train_cifar, batch_size=32, epochs=10, validation_data=test_cifar)

# save the weights
cnn_cifar_model.save_weights('./checkpoints/cnn_cifar_model/')



In [ ]:
### CIFAR ON RESNET50 ###

# set up the model
resnet_cifar_model = ml.ResNetModel()

resnet_cifar_model.compile(loss='categorical_crossentropy',
                        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                        metrics=['categorical_accuracy'])


# train the model
history = resnet_cifar_model.fit(train_cifar, batch_size=32, epochs=10, validation_data=test_cifar)

# save the weights
resnet_cifar_model.save_weights('./checkpoints/resnet_cifar_model/')



##ECO

In [ ]:
### LOAD THE ECO DATASET ###
drive.mount('/content/drive', force_remount=True)
os.chdir ("drive/MyDrive")

filename = "/content/drive/MyDrive/miniecoset_100obj_64px.h5"

In [ ]:
############ LOADING ECO DATASET ############

with h5py.File(filename, "r") as f:
    # get all the keys to explore further
    print(f.keys()) 
    # extracting data
    x_train_eco = f['train']['data'][()]
    y_train_eco = f['train']['labels'][()]
    x_test_eco = f['test']['data'][()]
    y_test_eco = f['test']['labels'][()]


# have same amount of train data as CIFAR (50.000)
x_train_eco = np.asarray([x_train_eco[index] for index in range(0, len(x_train_eco), 5)])
y_train_eco = np.asarray([y_train_eco[index] for index in range(0, len(y_train_eco), 5)])

# convert to a dataset
trainer_eco = tf.data.Dataset.from_tensor_slices((x_train_eco, y_train_eco))
tester_eco = tf.data.Dataset.from_tensor_slices((x_test_eco, y_test_eco))

# preprocess
train_eco = dpl.eco_data_pip(trainy, 'eco', 32)
test_eco = dpl.eco_data_pip(testy, 'eco', 32)


In [ ]:
### ECO ON CNN ###

# set up the model
cnn_eco_model = ml.ConvModel()

cnn_eco_model.compile(loss='categorical_crossentropy',
                        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                        metrics=['categorical_accuracy'])


# train the model
history = cnn_eco_model.fit(train_eco, batch_size=32, epochs=10, validation_data=test_eco)

# save the weights
cnn_eco_model.save_weights('./checkpoints/cnn_eco_model/')



In [ ]:
### ECO ON RESNET50 ###

# set up the model
resnet_eco_model = ml.ResNetModel()

resnet_eco_model.compile(loss='categorical_crossentropy',
                        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                        metrics=['categorical_accuracy'])


# train the model
history = resnet_eco_model.fit(train_eco, batch_size=32, epochs=10, validation_data=test_eco)

# save the weights
resnet_eco_model.save_weights('./checkpoints/resnet_eco_model/')

